<a href="https://colab.research.google.com/github/adoc2002/AdvDeepLearn/blob/main/ADL_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-09 21:53:11--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2023-12-09 21:53:11 (15.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
with open('input.txt', 'r',encoding='utf-8') as f:
  text = f.read()

In [4]:
print("length of the dataset in characters:", len(text))

length of the dataset in characters: 1115394


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [stoi[c] for c in s]
decode=lambda l: ''.join([itos[i]for i in l])
print(encode("Hello"))
print(decode(encode("Hello")))

[20, 43, 50, 50, 53]
Hello


In [8]:
import torch  #Change the first 1000 words into integers
data=torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
n= int(0.9*len(data)) #Train 90 percent of the data
train_data =data[:n]
val_data=data[n:]

In [10]:
block_size = 10  # Train in blocks for computational efficiency--> Max context length for predicitions
train_data[:block_size+1] # Have to go +1 since block size = 10 --> Want next prediction after 10


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64])

In [11]:
x=train_data[:block_size] # inputs to transformer
y=train_data[1:block_size+1] # next block size chars
for t in range(block_size): # iterate over block size of 10
  context=x[:t+1]
  target=y [t]
  print(f"When input is {context}, the target is:{target}")

When input is tensor([18]), the target is:47
When input is tensor([18, 47]), the target is:56
When input is tensor([18, 47, 56]), the target is:57
When input is tensor([18, 47, 56, 57]), the target is:58
When input is tensor([18, 47, 56, 57, 58]), the target is:1
When input is tensor([18, 47, 56, 57, 58,  1]), the target is:15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is:47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is:58
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]), the target is:47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]), the target is:64


In [12]:
torch.manual_seed(1337)
batch_size=5 # independent sequences to process in parallel
block_size =10

def get_batch(split):
  data=train_data if split =='train' else val_data #define data--> train else test
  ix=torch.randint(len(data)- block_size, (batch_size,)) # random offsets of batch size 5
  x= torch.stack([data[i:i+block_size] for i in ix])
  y= torch.stack([data[i+1:i+block_size+1]for i in ix])
  return x,y

xb,yb =get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target=   yb[b,t]
    print(f"When input is{context.tolist()} the target:{target}")

inputs:
torch.Size([5, 10])
tensor([[56, 41, 46,  1, 53,  5,  1, 58, 46, 43],
        [14, 33, 15, 23, 21, 26, 19, 20, 13, 25],
        [58, 46, 43, 56, 10,  0, 14, 59, 58,  1],
        [47, 60, 43,  1, 51, 43,  1, 58, 46, 63],
        [54, 53, 50, 50, 53,  1, 54, 50, 39, 63]])
targets:
torch.Size([5, 10])
tensor([[41, 46,  1, 53,  5,  1, 58, 46, 43,  1],
        [33, 15, 23, 21, 26, 19, 20, 13, 25, 10],
        [46, 43, 56, 10,  0, 14, 59, 58,  1, 21],
        [60, 43,  1, 51, 43,  1, 58, 46, 63,  1],
        [53, 50, 50, 53,  1, 54, 50, 39, 63, 57]])
----
When input is[56] the target:41
When input is[56, 41] the target:46
When input is[56, 41, 46] the target:1
When input is[56, 41, 46, 1] the target:53
When input is[56, 41, 46, 1, 53] the target:5
When input is[56, 41, 46, 1, 53, 5] the target:1
When input is[56, 41, 46, 1, 53, 5, 1] the target:58
When input is[56, 41, 46, 1, 53, 5, 1, 58] the target:46
When input is[56, 41, 46, 1, 53, 5, 1, 58, 46] the target:43
When input is[56, 41

In [13]:
import torch
torch.cuda.is_available()

False

In [14]:
print(xb) #input to transformer

tensor([[56, 41, 46,  1, 53,  5,  1, 58, 46, 43],
        [14, 33, 15, 23, 21, 26, 19, 20, 13, 25],
        [58, 46, 43, 56, 10,  0, 14, 59, 58,  1],
        [47, 60, 43,  1, 51, 43,  1, 58, 46, 63],
        [54, 53, 50, 50, 53,  1, 54, 50, 39, 63]])


In [15]:
print(vocab_size)

65


In [16]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):

    logits=self.token_embedding_table(idx) #(B,T,C)--> Batch = 5, Time = 10, Channel = vocab_size = 65
    if targets is None:
      loss=None
    else:
      B, T, C= logits.shape
      logits=logits.view(B*T, C) #changes shape of tensor for PyTorch
      targets=targets.view(B*T)
      loss = F.cross_entropy(logits, targets) # loss = cross entropy on logits and targets

    return logits,loss

  def generate(self, idx, max_new_tokens):  #idx is (B,T) array of indices in the current context--> Creates B, T--> B, T+1--> B, T+2--> to max new tokens
    for _ in range(max_new_tokens):
      logits,loss = self(idx) # get the predictions
      logits=logits[:,-1,:]  # focus only on the last time step
      probs=F.softmax(logits, dim=1)  # apply softmax to get probabilities
      idx_next=torch.multinomial(probs, num_samples=1) #sample from the distribution
      idx= torch.cat((idx, idx_next), dim=1) #append sampled index to the running sequence
    return idx




m= BigramLanguageModel(vocab_size)
out=m(xb,yb)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)


print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long), max_new_tokens=100)[0].tolist()))



torch.Size([50, 65])
tensor(4.5596, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [17]:
logits

tensor([[-0.6722,  0.2322, -0.1632,  ...,  0.1390,  0.7560,  0.4296],
        [-1.1895, -0.8407,  0.3134,  ...,  0.6707, -0.5969,  0.7567],
        [ 1.0901,  0.2170, -2.9996,  ..., -0.5472, -0.8017,  0.7761],
        ...,
        [-0.7470, -1.4852,  0.1714,  ...,  2.2019,  2.4498,  0.6347],
        [ 1.1513,  1.0539,  3.4105,  ..., -0.5686,  0.9079, -0.1701],
        [-0.8109,  0.2410, -0.1139,  ...,  1.4509,  0.1836,  0.3064]],
       grad_fn=<ViewBackward0>)

In [24]:
# Now want to train the model so will make optimizer
optimizer=torch.optim.AdamW(m.parameters(), lr=1e-3) # Used AdamW as optimizer--->


In [26]:
batch_size=32
for steps in range(1000):
  xb,yb= get_batch('train') # sample a batch of data
  logits,loss= m(xb, yb)
  optimizer.zero_grad(set_to_none=True) # Zero out gradients from previous step
  loss.backward() # get gradients for parameters
  optimizer.step() # use gradients to update the parameters

print(loss.item())



2.7870359420776367


In [32]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long), max_new_tokens=100)[0].tolist()))



YBYQ? cHues$zbeede'AU?$d
N3UNoicord&vWVk dgn biriKIINingXNCELQluls OMABToug;AU,& mes,

YM:CUSlico.



In [21]:
# Big improvement from line 16 results !!!

In [34]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long), max_new_tokens=400)[0].tolist()))


WprJBwfimy sTh bryano o,
PSe.
Ifr g-ur fr, RCHe sEDIlinQJHNPuthaHZ? cerd thirmTonl ba
o I YCI'd
M:
ut-
in o umDUZzImbllgry todshid
WIO,p ck:pm to ag wh

SVOOLAP.Prd, maRSSPote akGS3Bwitason my;f INove qa-bory avVjFo s mzCNN uf laHERSoohasERwe, thFXUMO s bakeqhond? y d bQRFay AngilaiendqAs, :PHoDin msur sshNCOJUzlurFqOR-nu

IBUve CADbe?p3f NCORomay'y ll, tNGCotenWG;Dq-thu of,lmV&vets wan!oZHosoXKWA


In [23]:
# Increased tokens to 400--> even better than line 29